In [1]:
# Start by importing the necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import pickle
import warnings
warnings.filterwarnings('ignore')
from pybaseball import schedule_and_record
from pybaseball import statcast
from pybaseball import playerid_lookup
from pybaseball import statcast_pitcher
from pybaseball import team_batting
from pybaseball import team_pitching
from pybaseball import batting_stats_range
from pybaseball import pitching_stats_range
from pybaseball import statcast_single_game
from pybaseball import playerid_reverse_lookup

In [2]:
test = pitching_stats_range('2021-04-01', '2021-04-05')
display(test)

,Name,Age,#days,Lev,Tm,G,GS,W,L,SV,...,StL,StS,GB/FB,LD,PU,WHIP,BAbip,SO9,SO/W,mlbID
1,Bryan Abreu,24,687,Maj-AL,Houston,2,0,1.0,NaN,NaN,...,0.18,0.07,0.62,0.08,0.08,0.692,0.154,2.1,1.0,650556
2,Jason Adam,29,687,Maj-NL,Chicago,2,0,NaN,NaN,NaN,...,0.13,0.13,0.67,0.17,0.00,3.000,0.400,10.8,1.0,592094
3,Jorge Alcala,25,688,Maj-AL,Minnesota,1,0,NaN,NaN,NaN,...,0.13,0.13,0.60,0.00,0.20,0.000,0.000,4.5,NaN,660896
4,Sandy Alcantara,25,691,Maj-NL,Miami,1,1,NaN,NaN,NaN,...,0.17,0.18,0.46,0.15,0.15,0.667,0.154,10.5,3.5,645261
5,Scott Alexander,31,687,Maj-NL,Los Angeles,2,0,NaN,NaN,NaN,...,0.15,0.00,0.63,0.00,0.13,0.000,0.000,0.0,NaN,518397
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
370,Huascar Ynoa,23,689,Maj-NL,Atlanta,1,0,NaN,NaN,NaN,...,0.07,0.00,0.50,0.25,0.00,1.000,0.250,0.0,NaN,660623
371,Alex Young,27,688,Maj-NL,Arizona,2,0,NaN,1.0,NaN,...,0.21,0.10,0.50,0.00,0.00,1.200,0.333,5.4,NaN,622065
372,T.J. Zeuch,25,688,Maj-AL,Toronto,1,1,NaN,NaN,NaN,...,0.17,0.10,0.54,0.23,0.00,1.000,0.231,2.3,1.0,643615
373,Kyle Zimmer,29,688,Maj-AL,Kansas City,2,0,NaN,NaN,NaN,...,0.16,0.08,0.57,0.14,0.14,0.818,0.143,9.8,2.0,622092


In [13]:
# This is the basic function that, in theory, should gather all of the data we are asking
all_play_data=statcast('2021-04-01','2021-06-23')

# The workaround:
# week1=pd.DataFrame([])
# for i in range(2,10):
#     day='2019-04-0'+str(i)
#     temp=pd.DataFrame([])
#     try:
#         temp=statcast(day,day)
#     except:
#         print(day)
#     if len(temp)>0:
#         week1=week1.append(pd.DataFrame(temp),ignore_index=True)
        
# Here is the imported the full dataframe for use going forward
# This file exceeded Github's upload limit and thus is not in the 
# repository. However, the above code shows how to source it.
# pickle_in=open("2019_data_june23.pickle","rb")
# every_pitch=pickle.load(pickle_in)

# This is another piece of code that will save tremendous amounts of runtime by doing it all
# at once.  This makes a dataframe of all active players and their identification numbers
# within different databases.
x=list(range(100000,800000))
all_players=playerid_reverse_lookup(x)
all_players=pd.DataFrame(all_players)
all_players=all_players[all_players.key_fangraphs!=-1]

This is a large query, it may take a moment to complete


100%|███████████████████████████████████████████| 84/84 [03:14<00:00,  2.32s/it]


Gathering player lookup table. This may take a moment.


In [48]:
def get_all_starters(data,lookback_start,lookback_end):
    dates=pd.date_range(lookback_start,lookback_end)
    all_starters={}
    for day in dates:
        day_starters=[]
        day_data=data[data.game_date==day]
        today_games=day_data.game_pk.unique()
        for game in today_games:
            game_stats=day_data[day_data.game_pk==game]
            home_counter=0
            away_counter=0
            l=game_stats.pitcher.value_counts().keys()
            for pitcher in l:
                m=game_stats[game_stats.pitcher==pitcher]
                m.reset_index(drop=True, inplace=True)
                if home_counter==0 and m.inning_topbot[0]=='Bot':
                    home_starter_id=pitcher
                    home_counter+=1
                elif away_counter==0 and m.inning_topbot[0]=='Top':
                    away_starter_id=pitcher
                    away_counter=+1
                else: 
                    None
            day_starters.append(home_starter_id)
            day_starters.append(away_starter_id)
        exit_date=day.strftime('%Y-%m-%d')
        all_starters.update({exit_date:day_starters})
    return all_starters

starters_on_day=get_all_starters(all_play_data,'2021-04-01','2021-06-23')

2021-04-01 00:00:00
2021-04-02 00:00:00
2021-04-03 00:00:00
2021-04-04 00:00:00
2021-04-05 00:00:00
2021-04-06 00:00:00
2021-04-07 00:00:00
2021-04-08 00:00:00
2021-04-09 00:00:00
2021-04-10 00:00:00
2021-04-11 00:00:00
2021-04-12 00:00:00
2021-04-13 00:00:00
2021-04-14 00:00:00
2021-04-15 00:00:00
2021-04-16 00:00:00
2021-04-17 00:00:00
2021-04-18 00:00:00
2021-04-19 00:00:00
2021-04-20 00:00:00
2021-04-21 00:00:00
2021-04-22 00:00:00
2021-04-23 00:00:00
2021-04-24 00:00:00
2021-04-25 00:00:00
2021-04-26 00:00:00
2021-04-27 00:00:00
2021-04-28 00:00:00
2021-04-29 00:00:00
2021-04-30 00:00:00
2021-05-01 00:00:00
2021-05-02 00:00:00
2021-05-03 00:00:00
2021-05-04 00:00:00
2021-05-05 00:00:00
2021-05-06 00:00:00
2021-05-07 00:00:00
2021-05-08 00:00:00
2021-05-09 00:00:00
2021-05-10 00:00:00
2021-05-11 00:00:00
2021-05-12 00:00:00
2021-05-13 00:00:00
2021-05-14 00:00:00
2021-05-15 00:00:00
2021-05-16 00:00:00
2021-05-17 00:00:00
2021-05-18 00:00:00
2021-05-19 00:00:00
2021-05-20 00:00:00


In [63]:
def team_abreviator(team,league):
    if team=='Atlanta':
        return 'ATL'
    elif team=='Arizona':
        return 'ARI'
    elif team=='Baltimore':
        return 'BAL'
    elif team=='Boston':
        return 'BOS'
    elif team=='Chicago':
        if league=='MLB-AL':
            return 'CWS'
        else:
            return 'CHC'
    elif team=='Cincinnati':
        return 'CIN'
    elif team=='Cleveland':
        return 'CLE'
    elif team=='Colorado':
        return 'COL'
    elif team=='Detroit':
        return 'DET'
    elif team=='Houston':
        return 'HOU'
    elif team=='Kansas City':
        return 'KC'
    elif team=='Los Angeles':
        if league=='MLB-AL':
            return 'LAA'
        else:
            return 'LAD'
    elif team=='Minnesota':
        return 'MIN'
    elif team=='Milwaukee':
        return 'MIL'
    elif team=='Miami':
        return 'MIA'
    elif team=='New York':
        if league=='MLB-AL':
            return 'NYY'
        else:
            return 'NYM'
    elif team=='Oakland':
        return 'OAK'
    elif team=='Pittsburgh':
        return 'PIT'
    elif team=='Philadelphia':
        return 'PHI'
    elif team=='San Diego':
        return 'SD'
    elif team=='San Francisco':
        return 'SF'
    elif team=='Seattle':
        return 'SEA'
    elif team=='St. Louis':
        return 'STL'
    elif team=='Texas':
        return 'TEX'
    elif team=='Tampa Bay':
        return 'TB'
    elif team=='Toronto':
        return 'TOR'
    elif team=='Washington':
        return 'WSH'
    else:
        print('Team not available in every pitch data')
        return 0
#####
def get_all_relievers(starters_on_day,data,lookback_start,lookback_end):
    dates=pd.date_range(lookback_start,lookback_end)
    dates=dates.astype(str)
    all_starters=[]
    for day in dates:
        for player in starters_on_day[day]:
            all_starters.append(player)
    f=pd.DataFrame(all_starters,columns=['starters'])
    f=f.starters.unique()
    g=pd.DataFrame(f,columns=['starters'])
    h=pd.merge(data,g,how='outer',left_on='Name',right_on='starters',indicator=True)
    relievers=h[h._merge!='both']
    return relievers
#####
batting_columns=['team','league','ab','runs','hits','doub','trip','hr','rbi','bb','avg','obp','slg',
                'est_ba_sa','est_woba_sa','sum_woba']
def recent_team_batting(data,every_pitch,start,end,team,league):
    data=data[data.iloc[:,4]==team]
    data=data[data.iloc[:,3]==league]
    ab=data.AB.sum()
    hits=data.H.sum()
    bb=data.BB.sum()+data.IBB.sum()+data.HBP.sum()
    doub=data['2B'].sum()
    trip=data['3B'].sum()
    hr=data.HR.sum()
    rbi=data.RBI.sum()
    avg=round(hits/ab,3)
    obp=round((hits+bb)/ab,3)
    slg=round((hits+doub+(trip*2)+(hr*3))/ab,3)
    runs=data.R.sum()
    data2=every_pitch[every_pitch.game_date<=end]
    data2=data2[data2.game_date>=start]
    team_abrev=team_abreviator(team,league)
    data3=data2[data2.home_team==team_abrev]
    data3=data3[data3.inning_topbot=='Bot']
    data4=data2[data2.away_team==team_abrev]
    data4=data4[data4.inning_topbot=='Top']
    data5=pd.concat([data3,data4])
    data5=data5.dropna(subset=['launch_angle', 'launch_speed', 'estimated_ba_using_speedangle'])
    est_ba_sa=data5.estimated_ba_using_speedangle.mean()
    est_woba_sa=data5.estimated_woba_using_speedangle.mean()
    sum_woba=data5.woba_value.sum() 
    df=pd.DataFrame([[team,league,ab,runs,hits,doub,trip,hr,rbi,bb,avg,obp,slg,
                     est_ba_sa,est_woba_sa,sum_woba]],columns=batting_columns)
    return df
#####
reliever_columns=['team_relief','league_relief','innings','hits','bb',
                      'k','at_bats','doub','trip','hr','era','ba',
                     'slg','obp','est_ba_sa','est_woba_sa','sum_woba']
def recent_bullpen(data,every_pitch,team,league,lookback_start,lookback_end):
    pen=data[data.Tm==team]
    pen=pen[pen.Lev==league]
    pen.IP=((pen.IP-round(pen.IP,0))*(10/3))+(round(pen.IP,0))
    innings=pen.IP.sum()
    hits=pen.H.sum()*9/innings
    bb=(pen.BB.sum()+pen.HBP.sum()+pen.IBB.sum())*9/innings
    k=pen.SO.sum()*9/innings
    at_bats=pen.AB.sum()*9/innings
    doub=pen['2B'].sum()*9/innings
    trip=pen['3B'].sum()*9/innings
    hr=pen.HR.sum()*9/innings
    era=pen.ER.sum()/innings*9
    ba=hits/at_bats
    slg=round((hits+doub+(trip*2)+(hr*3))/at_bats,3)
    obp=round((hits+bb)/(at_bats+bb),3)
    data2=every_pitch[every_pitch.game_date<=lookback_end]
    data2=data2[data2.game_date>=lookback_start]
    team_abrev=team_abreviator(team,league)
    data3=data2[data2.home_team==team_abrev]
    data3=data3[data3.inning_topbot=='Bot']
    data4=data2[data2.away_team==team_abrev]
    data4=data4[data4.inning_topbot=='Top']
    data5=pd.concat([data3,data4])
    data5=data5.dropna(subset=['launch_angle', 'launch_speed', 'estimated_ba_using_speedangle'])
    est_ba_sa=data5.estimated_ba_using_speedangle.mean()
    est_woba_sa=data5.estimated_woba_using_speedangle.mean()
    sum_woba=data5.woba_value.sum()/innings                  
    df=pd.DataFrame([[team,league,innings,hits,bb,
                      k,at_bats,doub,trip,hr,era,ba,
                     slg,obp,est_ba_sa,est_woba_sa,sum_woba]],columns=reliever_columns)
    return df
#####
def get_day_game_data(date,lookback_days):
    # initiate variables
    t1=datetime.datetime.now()
    game_ids=[]                      
    all_pitching_stats=[]
    all_batting_stats=[]
    data=pd.DataFrame([])
    fails=pd.DataFrame([])
    data2=pd.DataFrame([])
    home_starter_stats=pd.DataFrame([])
    away_starter_stats=pd.DataFrame([])
    home_batting_stats=pd.DataFrame([],columns=batting_columns)
    away_batting_stats=pd.DataFrame([],columns=batting_columns)
    home_reliever_stats=pd.DataFrame([],columns=reliever_columns)
    away_reliever_stats=pd.DataFrame([],columns=reliever_columns)
    all_starting_pitchers=[]
    #calculate necessary dates
    lookback_end=str(pd.to_datetime(date)-pd.to_timedelta(1,unit='D'))[:10]
    lookback_start=str(pd.to_datetime(date)-pd.to_timedelta(lookback_days+1,unit='D'))[:10]
    today_games=statcast(start_dt=date,end_dt=date)
    # Gather some of the necessary aggregate data
    all_pitching_stats=pitching_stats_range(lookback_start, lookback_end)
    all_pitching_stats.Name=all_pitching_stats.Name.str.lower()
    all_reliever_stats=get_all_relievers(starters_on_day,all_pitching_stats,lookback_start,
                                         lookback_end)
    all_batting_stats=batting_stats_range(lookback_start, lookback_end)
    game_ids=today_games.game_pk.unique()
    game_ids=game_ids.astype(int)
    # This deals with the occasional occurance of double headers
    double_header_count={'BOS':0,'MIL':0,'PIT':0,'MIA':0,'ATL':0,'PHI':0,
                      'CIN':0,'TOR':0,'ARI':0,'TEX':0,'OAK':0,'SF':0,
                      'LAD':0,'SD':0,'WSN':0,'NYM':0,'COL':0,'KC':0,
                      'CHW':0,'HOU':0,'BAL':0,'DET':0,'MIN':0,'CLE':0,
                      'NYY':0,'CHC':0,'STL':0,'BOS':0,'TB':0,'TB':0,
                      'LAA':0,'SEA':0}
    # Within this for loop is all of the processes for gathering the data on each particular game
    for game in game_ids:
        #retrieve individual game stats
        # There are some games that, for whatever reason, don't load properly.
        # The try and except statements handle those and keep a log of the games that
        # failed to load so that I can go back and gather that data manually        
        try:
            game_stats=statcast_single_game(game)
        except:
            print('game data pull failed')
            fails=fails.append(pd.DataFrame({'game#':game,'Date':date,'Home':0,'Away':0,
                                             'Reason':'game data pull failed'},index=[0]),ignore_index=True)
            continue
        if len(game_stats)==0:
            print('No game stats')
            fails=fails.append(pd.DataFrame({'game#':game,'Date':date,'Home':0,'Away':0,
                                             'Reason':'no game data'},index=[0]),ignore_index=True)
            continue
        else:
            None
        home,away=game_stats.home_team[0],game_stats.away_team[0]
        home_counter=0
        away_counter=0
        l=game_stats.pitcher.value_counts().keys()
        #determine 'starter' by who threw the most pitches. Normally this would simply be the 
        #pitchers who were in the first inning but with the rise of 'bullpenning' this is a work-around
        for pitcher in l:
            m=game_stats[game_stats.pitcher==pitcher]
            m.reset_index(drop=True, inplace=True)
            if home_counter==0 and m.inning_topbot[0]=='Top':
                home_starter_id=pitcher
                home_counter+=1
            elif away_counter==0 and m.inning_topbot[0]=='Bot':
                away_starter_id=pitcher
                away_counter=+1
            else: 
                None
        home_holder=all_players[all_players.key_mlbam==home_starter_id]
        home_holder.reset_index(drop=True,inplace=True)
        home_starter_name=str(home_holder.name_first[0])+' '+str(home_holder.name_last[0])
        away_holder=all_players[all_players.key_mlbam==away_starter_id]
        away_holder.reset_index(drop=True,inplace=True)
        away_starter_name=str(away_holder.name_first[0])+' '+str(away_holder.name_last[0])
        # This set of if statements handles cases where a starting pitcher does not have sufficient recent
        # data to be useful in the model. Such as not having pitched in a while or the rare case where there
        # are two pitchers with the same name.       
        if len(all_pitching_stats[all_pitching_stats.Name.str.contains(home_holder.name_last[0],regex=False)])==0:
            print('game#: ',game,' Date: ',date,' Home: ',home,' Away: ',away,'home pitcher with insuffucicient history')
            fails=fails.append(pd.DataFrame({'game#':game,'Date':date,'Home':home,'Away':away,
                                             'Reason':'home pitcher with insuffucicient history'},
                                            index=[0]),ignore_index=True)
            continue
        elif len(all_pitching_stats[all_pitching_stats.Name.str.contains(home_holder.name_last[0],regex=False)])==1:
            home_starter_stats=home_starter_stats.append(pd.DataFrame(all_pitching_stats[all_pitching_stats.Name.str.contains(home_holder.name_last[0],regex=False)]))
        elif len(all_pitching_stats[all_pitching_stats.Name==home_starter_name])==1:
            home_starter_stats=home_starter_stats.append(pd.DataFrame(all_pitching_stats[all_pitching_stats.Name==home_starter_name]))
        else:
            print('game#: ',game,' Date: ',date,' Home: ',home,' Away: ',away,'home pitcher with duplicate name?')
            fails=fails.append(pd.DataFrame({'game#':game,'Date':date,'Home':home,'Away':away,
                                             'Reason':'home pitcher with duplicate name?'},
                                            index=[0]),ignore_index=True)
            None
        if len(all_pitching_stats[all_pitching_stats.Name.str.contains(away_holder.name_last[0],regex=False)])==0:
            print('game#: ',game,' Date: ',date,' Home: ',home,' Away: ',away,'away pitcher with insuffucicient history')
            fails=fails.append(pd.DataFrame({'game#':game,'Date':date,'Home':home,'Away':away,
                                             'Reason':'away pitcher with insuffucicient history'},
                                            index=[0]),ignore_index=True)
            continue
        elif len(all_pitching_stats[all_pitching_stats.Name.str.contains(away_holder.name_last[0],regex=False)])==1:
            away_starter_stats=away_starter_stats.append(pd.DataFrame(all_pitching_stats[all_pitching_stats.Name.str.contains(away_holder.name_last[0],regex=False)]))
        elif len(all_pitching_stats[all_pitching_stats.Name==away_starter_name])==1:
            away_starter_stats=away_starter_stats.append(pd.DataFrame(all_pitching_stats[all_pitching_stats.Name==away_starter_name]))
        else:
            print('game#: ',game,' Date: ',date,' Home: ',home,' Away: ',away,'away pitcher with duplicate name?')
            fails=fails.append(pd.DataFrame({'game#':game,'Date':date,'Home':home,'Away':away,
                                             'Reason':'away pitcher with duplicate name?'},
                                            index=[0]),ignore_index=True)
            None
        # In different databases, there are a couple teams with different abreviations: this handles that.
        if home=='CWS':
            home='CHW'
        else:
            None
        if away=='CWS':
            away='CHW'
        else:
            None
        if home=='WSH':
            home='WSN'
        else:
            None
        if away=='WSH':
            away='WSN'
        else:
            None
        print(home,away)
        # This section is a workaround resulting because one database does not account for
        # extra inning games and leaves such games as a tie. To get around this I had to call
        # a different database and determine the winner by looking at the change in the team's
        # record after the game.
        home_schedule_record=schedule_and_record(pd.to_datetime(date).year,home)
        away_schedule_record=schedule_and_record(pd.to_datetime(date).year,away)
        home_schedule_record.set_index('Date',inplace=True)
        away_schedule_record.set_index('Date',inplace=True)
        y,y2,y3=pd.to_datetime(date),pd.to_datetime(lookback_end),pd.to_datetime(lookback_start)
        z,z2,z3=y.day,y2.day,y3.day
        date_string,date_string2,date_string3=str(y.strftime('%A, %b '))+str(z),str(y2.strftime('%A, %b '))+str(z2),str(y3.strftime('%A, %b '))+str(z3)
        date_string4=str(date_string)+str(' (1)')
        date_string5=str(date_string)+str(' (2)')
        
        if any(item==date_string for item in home_schedule_record.index):
            home_score=home_schedule_record.loc[date_string].R
            away_score=home_schedule_record.loc[date_string].RA
        else:
            if sum([item==date_string for item in home_schedule_record.index])==0 and double_header_count[home]==0:
                home_score=home_schedule_record.loc[date_string4].R
                away_score=home_schedule_record.loc[date_string4].RA
                double_header_count[home]=1
            else:
                home_score=home_schedule_record.loc[date_string5].R
                away_score=home_schedule_record.loc[date_string5].RA
                           
        # determine winner    
        if home_score>away_score:
            home_win=1
        elif home_score<away_score:
            home_win=0
        else:
            home_win=-99
        if any(item==date_string2 for item in home_schedule_record.index):
            home_record=home_schedule_record.loc[date_string2]['W-L']
        else:
            for i in range(1,10):
                y4=y2-pd.to_timedelta(i,unit='D')
                y4z=y4.day
                y4zdate=str(y4.strftime('%A, %b '))+str(y4z)
                if any(item==y4zdate for item in away_schedule_record.index):
                    home_record=away_schedule_record.loc[y4zdate]['W-L']
                    break
                else:
                    None
        if any(item==date_string3 for item in home_schedule_record.index):
            home_record_lookback=home_schedule_record.loc[date_string3]['W-L']
        else:
            for i in range(1,10):
                y4=y3-pd.to_timedelta(i,unit='D')
                y4z=y4.day
                y4zdate=str(y4.strftime('%A, %b '))+str(y4z)
                if any(item==y4zdate for item in away_schedule_record.index):
                    home_record_lookback=away_schedule_record.loc[y4zdate]['W-L']
                    break
                else:
                    None
        if any(item==date_string2 for item in away_schedule_record.index):
            away_record=away_schedule_record.loc[date_string2]['W-L']
        else:
            for i in range(1,10):
                y4=y2-pd.to_timedelta(i,unit='D')
                y4z=y4.day
                y4zdate=str(y4.strftime('%A, %b '))+str(y4z)
                if any(item==y4zdate for item in away_schedule_record.index):
                    away_record=away_schedule_record.loc[y4zdate]['W-L']
                    break
                else:
                    None
        if any(item==date_string3 for item in away_schedule_record.index):
            away_record_lookback=away_schedule_record.loc[date_string3]['W-L']
        else:
            for i in range(1,10):
                y4=y3-pd.to_timedelta(i,unit='D')
                y4z=y4.day
                y4zdate=str(y4.strftime('%A, %b '))+str(y4z)
                if any(item==y4zdate for item in away_schedule_record.index):
                    away_record_lookback=away_schedule_record.loc[y4zdate]['W-L']
                    break
                else:
                    None
        # This section determines the season win percentage for each team        
        home_current_wins,home_current_losses=int(home_record.split('-')[0]),int(home_record.split('-')[1])
        away_current_wins,away_current_losses=int(away_record.split('-')[0]),int(away_record.split('-')[1])
        home_pct=home_current_wins/(home_current_wins+home_current_losses)
        away_pct=away_current_wins/(away_current_wins+away_current_losses)        
        # This section determines the recent win percentage for each team
        home_lookback_wins,home_lookback_losses=int(home_record_lookback.split('-')[0]),int(home_record_lookback.split('-')[1])
        away_lookback_wins,away_lookback_losses=int(away_record_lookback.split('-')[0]),int(away_record_lookback.split('-')[1])
        home_recent_wins,home_recent_losses=home_current_wins-home_lookback_wins,home_current_losses-home_lookback_losses
        away_recent_wins,away_recent_losses=away_current_wins-away_lookback_wins,away_current_losses-away_lookback_losses
        home_streak=home_recent_wins/(home_recent_wins+home_recent_losses)
        away_streak=away_recent_wins/(away_recent_wins+away_recent_losses)
        # This section gathers some advanced statistics about the starting pitchers
        home_starter_adv=statcast_pitcher(lookback_start, lookback_end,int(home_starter_id))
        if len(home_starter_adv)==0:
            print('No home starter advanced stats')
            fails=fails.append(pd.DataFrame({'game#':game,'Date':date,'Home':home,'Away':away,
                                             'Reason':'No home starter advanced stats'},index=[0]),ignore_index=True)
            continue
        else:
            None
        home_starter_launch=home_starter_adv.launch_speed.mean()
        home_starter_adv = home_starter_adv.dropna(subset=['launch_angle', 'launch_speed', 'estimated_ba_using_speedangle'])
        home_starter_est_ba_sa=home_starter_adv.estimated_ba_using_speedangle.mean()
        home_starter_est_woba_sa=home_starter_adv.estimated_woba_using_speedangle.mean()
        home_starter_sum_woba=home_starter_adv.woba_value.sum()
        away_starter_adv=statcast_pitcher(lookback_start, lookback_end,int(away_starter_id))
        if len(away_starter_adv)==0:
            print('No away starter advanced stats')
            fails=fails.append(pd.DataFrame({'game#':game,'Date':date,'Home':home,'Away':away,
                                             'Reason':'No away starter advanced stats'},index=[0]),ignore_index=True)
            continue
        else:
            None
        away_starter_launch=away_starter_adv.launch_speed.mean()
        away_starter_adv = away_starter_adv.dropna(subset=['launch_angle', 'launch_speed', 'estimated_ba_using_speedangle'])
        away_starter_est_ba_sa=away_starter_adv.estimated_ba_using_speedangle.mean()
        away_starter_est_woba_sa=away_starter_adv.estimated_woba_using_speedangle.mean()
        away_starter_sum_woba=away_starter_adv.woba_value.sum()
        data=data.append(pd.DataFrame({'home_win':home_win,'home_score':home_score,
                                       'away_score':away_score,'date':date,'lookback_days':lookback_days,
                                       'home_team':home,'away_team':away,'home_pct':home_pct,'away_pct':away_pct,
                                       'home_streak':home_streak,'away_streak':away_streak,
                                       'home_starter_name':home_starter_name,
                                       'away_starter_name':away_starter_name,'home_starter_id':home_starter_id,
                                       'away_starter_id':away_starter_id,'home_starter_launch':home_starter_launch,
                                       'home_starter_est_ba_sa':home_starter_est_ba_sa,
                                       'home_starter_est_woba_sa':home_starter_est_woba_sa,
                                       'home_starter_sum_woba':home_starter_sum_woba,
                                      'away_starter_launch':away_starter_launch,
                                      'away_starter_est_ba_sa':away_starter_est_ba_sa,
                                       'away_starter_est_woba_sa':away_starter_est_woba_sa,
                                       'away_starter_sum_woba':away_starter_sum_woba},index=[0]),ignore_index=True)
        all_starting_pitchers.append(home_starter_name)
        all_starting_pitchers.append(away_starter_name)
        home_starter_stats.reset_index(drop=True,inplace=True)
        away_starter_stats.reset_index(drop=True,inplace=True)
        # This section calls the other functions and gathers data about each team
        home_batting=recent_team_batting(all_batting_stats,every_pitch,lookback_start, lookback_end,home_starter_stats.Tm[len(home_starter_stats)-1],home_starter_stats.Lev[len(home_starter_stats)-1])
        home_batting_stats=home_batting_stats.append((pd.DataFrame(home_batting)))
        away_batting=recent_team_batting(all_batting_stats,every_pitch,lookback_start, lookback_end,away_starter_stats.Tm[len(away_starter_stats)-1],away_starter_stats.Lev[len(away_starter_stats)-1])
        away_batting_stats=away_batting_stats.append((pd.DataFrame(away_batting)))
        home_relief=recent_bullpen(all_reliever_stats,every_pitch,home_starter_stats.Tm[len(home_starter_stats)-1],
                                   home_starter_stats.Lev[len(home_starter_stats)-1],lookback_start,lookback_end)
        home_reliever_stats=home_reliever_stats.append((pd.DataFrame(home_relief)))
        away_relief=recent_bullpen(all_reliever_stats,every_pitch,away_starter_stats.Tm[len(away_starter_stats)-1],
                                   away_starter_stats.Lev[len(away_starter_stats)-1],lookback_start,lookback_end)
        away_reliever_stats=away_reliever_stats.append((pd.DataFrame(away_relief)))
    # This section joins all of the data into a single dataframe and renames many of the columns for easier understanding
    data['home_join']=data.home_starter_name.str.replace(' ','')
    data['away_join']=data.away_starter_name.str.replace(' ','')
    home_starter_stats.columns=['hs'+str(col) for col in home_starter_stats.columns]
    away_starter_stats.columns=['as'+str(col) for col in away_starter_stats.columns]
    home_batting_stats.columns=['home_bat_'+str(col) for col in home_batting_stats.columns]
    away_batting_stats.columns=['away_bat_'+str(col) for col in away_batting_stats.columns]
    home_reliever_stats.columns=['homepen_'+str(col) for col in home_reliever_stats.columns]
    away_reliever_stats.columns=['awaypen_'+str(col) for col in away_reliever_stats.columns]
    home_starter_stats['hsjoin']=home_starter_stats.hsName.str.replace(' ','')
    away_starter_stats['asjoin']=away_starter_stats.asName.str.replace(' ','')
    data=data.merge(home_starter_stats,left_on='home_join',right_on='hsjoin')
    data=data.merge(away_starter_stats,left_on='away_join',right_on='asjoin')
    data=data.merge(home_batting_stats,how='left',left_on=['hsTm','hsLev'],right_on=['home_bat_team','home_bat_league'])
    data=data.merge(away_batting_stats,how='left',left_on=['asTm','asLev'],right_on=['away_bat_team','away_bat_league'])
    data=data.merge(home_reliever_stats,how='left',left_on=['hsTm','hsLev'],right_on=['homepen_team_relief','homepen_league_relief'])
    data=data.merge(away_reliever_stats,how='left',left_on=['asTm','asLev'],right_on=['awaypen_team_relief','awaypen_league_relief'])
    data.drop(['hsTm','asTm','away_join','home_join','asName','as#days',
                'asAge','asLev','hsName','hs#days','hsAge','hsLev','hsjoin',
                'asjoin','homepen_team_relief','homepen_league_relief',
               'awaypen_team_relief','awaypen_league_relief'],axis=1,inplace=True)
    data=data.drop_duplicates()
    return data,fails
#####
def get_game_data_range(start,end,lookback_days):
    dates=pd.date_range(start,end)
    dates=dates.astype(str)
    data=pd.DataFrame([])
    fails=pd.DataFrame([])
    for day in dates:
        day_data,day_fails=get_day_game_data(day,lookback_days)
        data=pd.concat([data,day_data],ignore_index=True)
        fails=pd.concat([fails,day_fails],ignore_index=True)
    return data,fails
game_data=get_game_data_range('2021-04-01','2021-06-23', 10)

This is a large query, it may take a moment to complete


100%|█████████████████████████████████████████████| 1/1 [00:10<00:00, 10.17s/it]


IndexError: list index out of range

In [64]:
test = statcast('2022-04-01','2022-04-01')

This is a large query, it may take a moment to complete


100%|█████████████████████████████████████████████| 1/1 [00:06<00:00,  6.65s/it]


In [67]:
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 225)
display(test)

,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,spin_dir,spin_rate_deprecated,break_angle_deprecated,break_length_deprecated,zone,des,game_type,stand,p_throws,home_team,away_team,type,hit_location,bb_type,balls,strikes,game_year,pfx_x,pfx_z,plate_x,plate_z,on_3b,on_2b,on_1b,outs_when_up,inning,inning_topbot,hc_x,hc_y,tfs_deprecated,tfs_zulu_deprecated,fielder_2,umpire,sv_id,vx0,vy0,vz0,ax,ay,az,sz_top,sz_bot,hit_distance_sc,launch_speed,launch_angle,effective_speed,release_spin_rate,release_extension,game_pk,pitcher.1,fielder_2.1,fielder_3,fielder_4,fielder_5,fielder_6,fielder_7,fielder_8,fielder_9,release_pos_y,estimated_ba_using_speedangle,estimated_woba_using_speedangle,woba_value,woba_denom,babip_value,iso_value,launch_speed_angle,at_bat_number,pitch_number,pitch_name,home_score,away_score,bat_score,fld_score,post_away_score,post_home_score,post_bat_score,post_fld_score,if_fielding_alignment,of_fielding_alignment,spin_axis,delta_home_win_exp,delta_run_exp
1563,SL,2022-04-01,85.1,-2.21,6.37,"Ginkel, Kevin",666624,656464,field_out,hit_into_play,<NA>,<NA>,<NA>,<NA>,9,"Christopher Morel grounds out, shortstop Ryan ...",S,R,R,AZ,CHC,X,6,ground_ball,1,1,2022,0.12,-0.4,0.3,1.58,<NA>,<NA>,<NA>,2,9,Top,113.1,150.57,<NA>,<NA>,492802,<NA>,<NA>,5.706718,-123.945836,-4.213896,0.200606,21.874919,-35.817016,3.32,1.51,24,88.0,-2,86.0,2427,6.6,707046,656464,492802,660681,671083,691021,669208,682630,678489,686527,53.92,0.229,0.219,0.0,<NA>,0,0,2,65,3,Slider,6,0,0,6,0,6,0,6,Standard,Standard,73,0.0,<NA>
1697,FF,2022-04-01,96.4,-2.29,6.41,"Ginkel, Kevin",666624,656464,NaN,ball,<NA>,<NA>,<NA>,<NA>,11,"Christopher Morel grounds out, shortstop Ryan ...",S,R,R,AZ,CHC,B,<NA>,NaN,0,1,2022,-0.76,1.34,-0.17,3.98,<NA>,<NA>,<NA>,2,9,Top,<NA>,<NA>,<NA>,<NA>,492802,<NA>,<NA>,7.418628,-140.222752,-3.982311,-11.664936,30.421936,-13.771808,3.76,1.78,<NA>,<NA>,<NA>,97.5,2338,6.8,707046,656464,492802,660681,671083,691021,669208,682630,678489,686527,53.67,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,65,2,4-Seam Fastball,6,0,0,6,0,6,0,6,Standard,Standard,222,0.0,<NA>
1829,SL,2022-04-01,84.6,-2.33,6.5,"Ginkel, Kevin",666624,656464,NaN,swinging_strike,<NA>,<NA>,<NA>,<NA>,8,"Christopher Morel grounds out, shortstop Ryan ...",S,R,R,AZ,CHC,S,<NA>,NaN,0,0,2022,0.26,-0.18,-0.14,1.87,<NA>,<NA>,<NA>,2,9,Top,<NA>,<NA>,<NA>,<NA>,492802,<NA>,<NA>,4.613384,-123.246871,-4.203181,1.927576,20.991363,-33.512179,3.32,1.51,<NA>,<NA>,<NA>,85.7,2316,6.6,707046,656464,492802,660681,671083,691021,669208,682630,678489,686527,53.88,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,65,1,Slider,6,0,0,6,0,6,0,6,Standard,Standard,43,0.0,<NA>
2030,SL,2022-04-01,84.7,-2.34,6.51,"Ginkel, Kevin",663891,656464,field_out,hit_into_play,<NA>,<NA>,<NA>,<NA>,3,Nelson Maldonado flies out to right fielder Do...,S,R,R,AZ,CHC,X,9,fly_ball,0,1,2022,0.11,-0.24,0.73,2.82,<NA>,<NA>,<NA>,1,9,Top,191.53,132.82,<NA>,<NA>,492802,<NA>,<NA>,6.976667,-123.248894,-1.836021,-0.194736,22.896219,-34.515887,3.3,1.5,232,80.5,43,85.5,2291,6.6,707046,656464,492802,660681,671083,691021,669208,682630,678489,686527,53.86,0.026,0.028,0.0,<NA>,0,0,3,64,2,Slider,6,0,0,6,0,6,0,6,Standard,Standard,48,0.001,<NA>
2100,SL,2022-04-01,84.5,-2.28,6.36,"Ginkel, Kevin",663891,656464,NaN,swinging_strike,<NA>,<NA>,<NA>,<NA>,9,Nelson Maldonado flies out to right fielder Do...,S,R,R,AZ,CHC,S,<NA>,NaN,0,0,2022,0.03,-0.36,0.45,1.59,<NA>,<NA>,<NA>,1,9,Top,<NA>,<NA>,<NA>,<NA>,492802,<NA>,<NA>,6.347346,-123.042863,-4.102931,-0.859313,22.527919,-35.290161,3.3,1.5,<NA>,<NA>,<NA>,85.3,2120,6.6,707046,656464,492802,660681,671083,691021,669208,682630,678489,686527,53.91,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,64,1,Slider,6,0,0,6,0,6,0,6,Standard,Standard,47,0.0,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,..

In [68]:
test2 = statcast_single_game(706814)

In [69]:
display(test2)

,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,spin_dir,spin_rate_deprecated,break_angle_deprecated,break_length_deprecated,zone,des,game_type,stand,p_throws,home_team,away_team,type,hit_location,bb_type,balls,strikes,game_year,pfx_x,pfx_z,plate_x,plate_z,on_3b,on_2b,on_1b,outs_when_up,inning,inning_topbot,hc_x,hc_y,tfs_deprecated,tfs_zulu_deprecated,fielder_2,umpire,sv_id,vx0,vy0,vz0,ax,ay,az,sz_top,sz_bot,hit_distance_sc,launch_speed,launch_angle,effective_speed,release_spin_rate,release_extension,game_pk,pitcher.1,fielder_2.1,fielder_3,fielder_4,fielder_5,fielder_6,fielder_7,fielder_8,fielder_9,release_pos_y,estimated_ba_using_speedangle,estimated_woba_using_speedangle,woba_value,woba_denom,babip_value,iso_value,launch_speed_angle,at_bat_number,pitch_number,pitch_name,home_score,away_score,bat_score,fld_score,post_away_score,post_home_score,post_bat_score,post_fld_score,if_fielding_alignment,of_fielding_alignment,spin_axis,delta_home_win_exp,delta_run_exp
42,NaN,2022-04-01,NaN,NaN,NaN,"Mayer, Marcelo",691785,680961,field_out,hit_into_play,NaN,NaN,NaN,NaN,NaN,Marcelo Mayer flies out to left fielder Jordan...,S,L,R,TB,BOS,X,7.0,fly_ball,0,0,2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,9,Top,72.81,97.98,NaN,NaN,666712,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,706814,680961,666712,657212,689172,695020,672517,682040,687071,680881,NaN,NaN,NaN,0.00,NaN,0.0,0.0,NaN,74,1,NaN,9,3,3,9,3,9,3,9,NaN,NaN,NaN,0.000,NaN
52,NaN,2022-04-01,NaN,NaN,NaN,"Rafaela, Ceddanne",678882,680961,strikeout,swinging_strike,NaN,NaN,NaN,NaN,NaN,Ceddanne Rafaela strikes out swinging.,S,R,R,TB,BOS,S,2.0,NaN,0,2,2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,9,Top,NaN,NaN,NaN,NaN,666712,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,706814,680961,666712,657212,689172,695020,672517,682040,687071,680881,NaN,NaN,NaN,0.00,NaN,0.0,0.0,NaN,73,3,NaN,9,3,3,9,3,9,3,9,NaN,NaN,NaN,0.001,NaN
59,NaN,2022-04-01,NaN,NaN,NaN,"Rafaela, Ceddanne",678882,680961,NaN,swinging_strike,NaN,NaN,NaN,NaN,NaN,Ceddanne Rafaela strikes out swinging.,S,R,R,TB,BOS,S,NaN,NaN,0,1,2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,9,Top,NaN,NaN,NaN,NaN,666712,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,706814,680961,666712,657212,689172,695020,672517,682040,687071,680881,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,73,2,NaN,9,3,3,9,3,9,3,9,NaN,NaN,NaN,0.000,NaN
69,NaN,2022-04-01,NaN,NaN,NaN,"Rafaela, Ceddanne",678882,680961,NaN,swinging_strike,NaN,NaN,NaN,NaN,NaN,Ceddanne Rafaela strikes out swinging.,S,R,R,TB,BOS,S,NaN,NaN,0,0,2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,9,Top,NaN,NaN,NaN,NaN,666712,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,706814,680961,666712,657212,689172,695020,672517,682040,687071,680881,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,73,1,NaN,9,3,3,9,3,9,3,9,NaN,NaN,NaN,0.000,NaN
76,NaN,2022-04-01,NaN,NaN,NaN,"Decker, Nick",678006,680961,field_out,hit_into_play,NaN,NaN,NaN,NaN,NaN,Nick Decker flies out to center fielder Shane ...,S,L,R,TB,BOS,X,8.0,fly_ball,0,0,2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,9,Top,106.97,59.05,NaN,NaN,666712,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,706814,680961,666712,657212,689172,695020,672517,682040,687071,680881,NaN,NaN,NaN,0.00,NaN,0.0,0.0,NaN,72,1,NaN,9,3,3,9,3,9,3,9,NaN,NaN,NaN,0.001,NaN
87,NaN,2022-04-01,NaN,NaN,NaN,"Plawecki, Kevin",608700,680961,home_run,hit_into_play,NaN,NaN,NaN,NaN,NaN,Kevin Plawecki homers (1) on a fly ball to lef...,S,R,R,TB,BOS,X,NaN,fly_ball,1,0,2022,NaN,NaN,NaN,NaN,NaN,NaN,686765.0,0,9,Top,55.33,50.31,NaN,NaN,666712,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,706814,680961,666712,657212,689172,695020,672517,682040,687071,680881,NaN,NaN,NaN,2.00,NaN,0.0,3.0,NaN,71,2,NaN,9,1,1,9,3,9,3,9,NaN,NaN,NaN,-0.001,NaN
107,NaN,2022-04-01,NaN,NaN,NaN,"Plawecki, Kevin",608700,680961,NaN,ball,NaN,NaN,NaN,NaN,NaN,Kevin Plawecki homers (1) on a fly ball to lef...,S,R,R,TB,BOS,B,NaN,NaN,0,0,2022,NaN,NaN,NaN,NaN,NaN,NaN,686765

In [58]:
# The section below transforms the dates into the form that matches the rest of this notebook
odds_data=pd.read_csv('mlb-odds-2021.csv')
# odds_data['month']=round(odds_data.Date/100).astype(int)
# odds_data['day']=(odds_data.Date-odds_data.month*100).astype(int)
# odds_data['game_day']=0

odds_data21=pd.read_csv('mlb-odds-2021.csv')

for i in range(len(odds_data)):
    year = "2021"
    month = ""
    day = ""
    uncleaned_date = str(odds_data.Date[i])
    if len(uncleaned_date) == 3:
        month = uncleaned_date[0:1]
        day = uncleaned_date[1:]
    else:
        month = uncleaned_date[0:2]
        day = uncleaned_date[2:]
    odds_data.Date[i] = year + "-" + month + "-" + day
    

for i in range(len(odds_data)):
    if odds_data.Team[i]=='SFO':
        odds_data.Team[i]='SF'
    elif odds_data.Team[i]=='WAS':
        odds_data.Team[i]='WSN'
    elif odds_data.Team[i]=='TAM':
        odds_data.Team[i]='TB'
    elif odds_data.Team[i]=='CWS':
        odds_data.Team[i]='CHW'
    elif odds_data.Team[i]=='KAN':
        odds_data.Team[i]='KC'
    elif odds_data.Team[i]=='CUB':
        odds_data.Team[i]='CHC'
    elif odds_data.Team[i]=='SDG':
        odds_data.Team[i]='SD'
    else:
        None
    
# This initiates the necessary variables
game_data['home_money_open']=None
game_data['home_money_close']=None
game_data['home_money_change']=None
game_data['away_money_open']=None
game_data['away_money_close']=None
game_data['away_money_change']=None
game_data['home_prob_open']=None
game_data['home_prob_close']=None
game_data['home_prob_change']=None
game_data['ou_open']=None
game_data['ou_close']=None   
# This function uses the money line odds to calculate the betting markets implied
# probablility of the home team winning 
def home_pct_chance(home_money,away_money):
    if home_money>0:
        a1=100/(home_money+100)
    else:
        a1=-home_money/(100-home_money)
    if away_money>0:
        a2=100/(away_money+100)
    else:
        a2=-away_money/(100-away_money)
    return(a1/(a1+a2))
# This for loop fills in the relevant betting related columns into the data frame
for i in range(len(game_data)):
    try:
        home=[]
        away=[]
        date=game_data.date[i]
        home_team=game_data.home_team[i]
        away_team=game_data.away_team[i]
        home=odds_data.loc[(odds_data['Date']==date) & (odds_data['Team']==home_team)]
        away=odds_data.loc[(odds_data['Date']==date) & (odds_data['Team']==away_team)]
        home.reset_index(drop=True,inplace=True)
        away.reset_index(drop=True,inplace=True)
        game_data.home_money_close[i]=int(home.Close[0])
        if home.Open[0]=='NL':
            game_data.home_money_open[i]=game_data.home_money_close[i]
            game_data.home_money_change[i]=0
        else:
            game_data.home_money_open[i]=int(home.Open[0])
            game_data.home_money_change[i]=int(home.Close[0])-int(home.Open[0]) 
        game_data.away_money_close[i]=int(away.Close[0])
        if away.Open[0]=='NL':
            game_data.away_money_open[i]=game_data.away_money_close[i]
            game_data.away_money_change[i]=0
        else:
            game_data.away_money_open[i]=int(away.Open[0])
            game_data.away_money_change[i]=int(away.Close[0])-int(away.Open[0]) 
        game_data.home_prob_open[i]=home_pct_chance(game_data.home_money_open[i],game_data.away_money_open[i])
        game_data.home_prob_close[i]=home_pct_chance(game_data.home_money_close[i],game_data.away_money_close[i])
        game_data.home_prob_change[i]=game_data.home_prob_close[i]-game_data.home_prob_open[i]
        game_data.ou_open[i]=home.OpenOU[0]
        game_data.ou_close[i]=away.CloseOU[0]
    except:
        None

In [59]:
game_data=game_data.fillna(0)
game_data=game_data[game_data.home_streak<1.001]
game_data=game_data[game_data.away_streak<1.001]
game_data=game_data[game_data.home_streak>-.001]
game_data=game_data[game_data.away_streak>-.001]
df=game_data.copy()
home_dummies=pd.get_dummies(df.home_team)
away_dummies=pd.get_dummies(df.away_team)
home_dummies.columns=['h_'+str(col) for col in home_dummies.columns]
away_dummies.columns=['a_'+str(col) for col in away_dummies.columns]
df=df.merge(home_dummies,left_index=True,right_index=True)
df=df.merge(away_dummies,left_index=True,right_index=True)
df.drop(['date','lookback_days','home_team','away_team','home_starter_name',
         'away_starter_name','home_starter_id','away_starter_id','home_bat_team','home_bat_league',
        'away_bat_team','away_bat_league'],axis=1,inplace=True)
# This moves the final moneyline to the last columns to make it easier to examine the real world application later on
df['home_money_close2']=df.home_money_close
df['away_money_close2']=df.away_money_close
df.drop(['home_money_close','away_money_close'],axis=1,inplace=True)
df['home_money_close']=df.home_money_close2
df['away_money_close']=df.away_money_close2
df.drop(['home_money_close2','away_money_close2'],axis=1,inplace=True)
df.reset_index(drop=True,inplace=True)
display(df.head())
# pickle_out=open("cleaned_data.pickle","wb")
# pickle.dump(df,pickle_out)
# pickle_out.close()

,home_win,home_score,away_score,home_pct,away_pct,home_streak,away_streak,home_starter_launch,away_starter_launch,hsG,...,a_SD,a_SEA,a_SF,a_STL,a_TB,a_TEX,a_TOR,a_WSN,home_money_close,away_money_close
0,0,1.0,9.0,0.300000,0.750000,0.375000,0.777778,86.685185,85.618000,2,...,0,0,0,0,1,0,0,0,0,0
1,0,1.0,15.0,0.700000,0.500000,0.625000,0.625000,84.233962,84.141667,2,...,0,0,0,0,0,0,0,1,0,0
2,1,7.0,2.0,0.545455,0.666667,0.625000,0.666667,82.107692,76.900000,1,...,0,0,0,0,0,0,0,0,0,0
3,0,1.0,3.0,0.333333,0.583333,0.333333,0.555556,86.094118,86.383333,2,...,1,0,0,0,0,0,0,0,0,0
4,1,9.0,6.0,0.600000,0.666667,0.500000,0.714286,82.367273,79.432143,2,...,0,0,0,0,0,0,0,0,0,0
